In [ ]:
import numpy as np
from tqdm import tqdm
import os
import sys
import json

## 1. 데이터 불러오기 

In [ ]:
# 학습 데이터를 불러와서 id, document, label 리스트를 반환
def read_train_data(file_path):
    with open(file_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()

    ids, docs, labels = [], [], []
    for index, line in enumerate(tqdm(lines, desc="read_data")):
        pieces = line.strip().split("\t")
        # 데이터의 형태가 올바른지 체크
        assert len(pieces) == 3
        if(index == 0):
            continue
        id, doc, label = pieces[0], pieces[1], int(pieces[2])
        ids.append(id)
        docs.append(doc)
        labels.append(label)

    return ids, docs, labels


# 평가 데이터를 불러와서 id, document 리스트를 반환
def read_test_data(file_path):
  with open(file_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()

  ids, docs = [], [] 
  for index, line in enumerate(tqdm(lines, desc="read_data")):
      pieces = line.strip().split("\t")
      # 데이터의 형태가 올바른지 체크
      assert len(pieces) == 2
      if(index == 0):
          continue
      id, doc = pieces[0], pieces[1]
      ids.append(id)
      docs.append(doc)

  return ids, docs

In [ ]:
root_dir = "drive/My Drive/Hackathon/"

train_dir = os.path.join(root_dir, "ratings_train.txt")
test_dir = os.path.join(root_dir, "ratings_test.txt")

train_id, train_doc, train_label = read_train_data(file_path=train_dir)
test_id, test_doc, test_label = read_train_data(file_path=test_dir)

## 2. 리뷰 데이터 형태소 분석 및 품사 태깅 
#### 1) konlpy 패키지의 Okt 클래스 사용

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
okt = Okt()

#### 2) 각 리뷰를 '형태소/품사' 형태로 토큰화

In [ ]:
def tokenize(doc):
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [ ]:
tokenize(train_doc[0])

In [ ]:
train_doc_tkn = [(tokenize(x)) for x in train_doc]
test_doc_tkn = [(tokenize(x)) for x in test_doc]

In [ ]:
# tokenize된 총 20만개의 학습 데이터셋
total_docs = np.concatenate((train_doc_tkn, test_doc_tkn), axis = 0)
total_labels = np.concatenate((train_label, test_label), axis = 0)

## 3. 모델 1 - CountVectorizer + TfidfTransformer + SVM

In [ ]:
# countvectorizer의 input 형태로 전환 
train_docs = []

for i in range(len(total_docs)):
  seq = ""
  for j in range(len(total_docs[i])):
    seq = seq + total_docs[i][j] + " "
  train_docs.append(seq)

#### 1) CountVectorizer를 이용하여 각 리뷰를 벡터화

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
feature_train_X = vectorizer.fit_transform(train_docs)

#### 2) TfidfTransformer를 이용하여 각 토큰의 tf-idf값으로 벡터를 transform

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(smooth_idf = True)
trans_train_X = transformer.fit_transform(feature_train_X)

#### 3) Support Vector Machine으로 학습
- 각 모델의 validation accuracy 비교

| Validation Accuracy | KNN | DT | MEM | NB | SVM | 
| --- | --- | --- | --- | --- | --- |
| | 76% | 75% | 83% | 83% | 85% |

- hyper-parameter tuning  
C=1.62, kernel='rbf'

In [ ]:
from sklearn.svm import SVC

model_1 = SVC(C=1.62, kernel='rbf')
model_1.fit(trans_train_X, total_labels)

In [ ]:
from sklearn.externals import joblib 

# 실제 테스트 상황에서 사용하기 위해 모델 저장 
joblib.dump(model_1, os.path.join(root_dir, "model_1.pkl"))

## 4. 모델 2 - Word2vec + SVM


#### 1) gensim 패키지의 Word2Vec 이용하여 각 단어의 벡터값 계산

In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=total_docs, size = 300, window=5, min_count=1, workers=4, sg=1)

In [ ]:
# 실제 테스트 상황에서 사용하기 위해 모델 저장 
w2v_model.save(os.path.join(root_dir, "w2v_model.model"))

#### 2) 각 리뷰를 단어들의 벡터값으로 표현

In [ ]:
# total_docs에는 tokenize된 데이터가, w2v_train_X에는 벡터화된 데이터가 들어감
# gensim의 wv.get_vector를 이용해 total_docs의 각 단어 벡터를 받아와 w2v_train_X에 저장

temp_lst, w2v_train_X = [], []

for i in range(len(total_docs)):
  temp_lst = []
  for j in range(len(total_docs[i])):
    temp_lst.append(w2v_model.wv.get_vector(total_docs[i][j]))
  w2v_train_X.append(temp_lst)  

#### 3) 열별 평균을 내어 각 리뷰 벡터를 300X1 벡터로 변환

In [ ]:
# w2v_train_X에는 행은 300, 열은 문장 내 단어 개수인 행렬이 20만개 들어가 있음
# 예를 들어 첫번째 문장의 단어수가 11개면 첫번째 행렬은 300x11
# 전처리 과정에서 단어 수가 0개인 문장이 생길 수 있음

#각 열별로 평균을 낸 300x1의 벡터를 구하여 total_X_lst에 저장
# total_X_lst에는 20만개의 문장 벡터가 들어감

total_X_lst, total_empty_lst, total_Y_lst = [], [], []

for i in range(len(w2v_train_X)):
  # 비어있는 array의 평균을 구할 때 뜨는 에러는 try-except문으로 처리
  try:
    temp = list(np.array(w2v_train_X[i]).mean(axis= 0))
    total_X_lst.append(temp)
    total_Y_lst.append(total_labels[i])
  except:
    total_empty_lst.append(i)

#### 4) Support Vector Machine학습
- Standard Scaling

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# standard scaling
model_2 = Pipeline([
            ("scaler", StandardScaler()),
            ("svm_clf", SVC(kernel = "rbf", C=1.62))])
model_2.fit(total_X_lst, total_Y_lst)

In [ ]:
# 실제 테스트 상황에서 사용하기 위해 모델 저장 
joblib.dump(model_2, os.path.join(root_dir, "model_2.pkl"))

## 5. 평가 데이터 예측

#### 1) 평가 데이터 불러오기

In [ ]:
test_id, test_doc = read_test_data(file_path=os.path.join(root_dir, "ratings_eval.txt"))

#### 2) 평가 데이터 전처리

In [ ]:
test_doc_tkn = [(tokenize(x)) for x in test_doc]

#### 3) 모델 1 prediction

In [ ]:
feature_test_X = vectorizer.transform(test_doc_tkn)
trans_test_X = transformer.fit_transform(feature_test_X)

md1_pred = model_1.predict(trans_tes_X)

#### 4) 모델 2 prediction

In [ ]:
temp_lst, test_word2vec_lst = [], []

for i in range(len(test_X)):
  temp_lst = []
  for j in range(len(test_X[i])):
    try:
      temp_lst.append(w2v_model.wv.get_vector(test_X[i][j]))
    except:
      pass
  test_word2vec_lst.append(temp_lst)

In [ ]:
test_X_lst, test_empty_lst, test_id_lst = [], [], []

for i in range(len(test_word2vec_lst)):
  # 비어있는 array의 평균을 구할 때 뜨는 에러 처리
  try:
    temp = list(np.array(test_word2vec_lst[i]).mean(axis= 0))
    test_X_lst.append(temp)
    test_id_lst.append(test_id[i])
  except:
    # 모델 2로 예측할 수 없는 리뷰의 index를 저장 
    test_empty_lst.append(i)

In [ ]:
md2_pred = model_2.predict(test_X_lst)

In [ ]:
id_pred_lst = []
for i in range(len(test_id_lst)):
  id_pred_lst.append([test_id_lst[i], md2_pred[i]])

In [ ]:
# 모델 2가 예측하지 못하는 리뷰의 라벨은 0.5로 채움
for i in test_empty_lst:
  id_pred_lst = id_pred_lst[:i] + [[test_id[i], 0.5]] + id_pred_lst[i:]

## 6. 앙상블

- 모델 1과 모델 2의 예측값에 각각 0.9, 1.1의 가중치를 두어 최종 예측값 산출

In [ ]:
ensemble_pred = []

for i in range(len(md1_pred)):
  md1_v = float(md1_pred[i]) * 0.9
  md2_v = float(id_pred_lst[i][1]) * 1.1
  pred = (md1_v + md2_v) / 2

  if pred > 0.5:
    ensemble_pred.append((id_pred_lst[i][0], 1))
  else:
    ensemble_pred.append((id_pred_lst[i][0], 0))